In [1]:
import pandas as pd

In [3]:
test = pd.read_excel("./Data_auxiliaires/MDB-INSEE-V2.xls")

WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero


In [4]:
# delete row with A or B in CODGEO
print(len(test["CODGEO"]))
test = test[~test["CODGEO"].str.contains("A|B")]

test["CODGEO"] = test["CODGEO"].astype(int)
print(len(test["CODGEO"]))

36677
36317


In [5]:
# drop object columns
print(len(test.columns))
for x in test.columns:
    if test[x].dtype == "object":
        test.drop(x, axis=1, inplace=True)
print(len(test.columns))

101
87


In [94]:
X = pd.read_csv("../Prix_metre/UX_avec_prix_et_images_complet_et_inflation.csv")

In [8]:
print(len(X["postal_code"]))
print(len(X["postal_code"].unique()))
print(len(test["CODGEO"]))
print(len(test["CODGEO"].unique()))

46707
4955
36317
36317


In [26]:
# print the column CODGEO and order by ascending
essai = test[["CODGEO"]].sort_values(by=["CODGEO"], ascending=True)
essai

,CODGEO
0,1001
1,1002
2,1004
3,1005
4,1006
...,...
36672,97420
36673,97421
36674,97422
36675,97423


In [9]:
X = X.merge(test, how='left', left_on='postal_code', right_on='CODGEO')
print(len(X))

46707


In [11]:
# print the number of duplicate rows in X_train
print(X[["id_annonce"]].duplicated().sum())

0


In [12]:
# drop duplicate rows
X = X.drop_duplicates(subset=["id_annonce"], keep="first")

In [27]:
# Nombres de valeurs manquantes
print(X.shape)
X.isnull().sum()

print(X[X[["Score PIB"]].isnull().all(axis=1)].shape)
# X[X[["Score PIB"]].isnull().all(axis=1)].to_csv("essai.csv",index=False)

(46707, 142)
(26551, 142)


PermissionError: [Errno 13] Permission denied: 'essai.csv'

# Implémentation du gros dataset de l'INSEE

In [ ]:
X.to_csv('X_07_01.csv', index=False)

In [99]:
insee_plein = pd.read_csv("C:/Users/maila/Documents/Centrale Nantes/EI2/INFOIA/STASC/Data_Challenge/insee-plein.csv",sep=";")

C:\Users\maila\AppData\Local\Temp\ipykernel_22984\2540055051.py:1: DtypeWarning: Columns (69,185,186,1486,1488,1501,1527) have mixed types. Specify dtype option on import or set low_memory=False.
  insee_plein = pd.read_csv("C:/Users/maila/Documents/Centrale Nantes/EI2/INFOIA/STASC/Data_Challenge/insee-plein.csv",sep=";")


In [100]:
print(insee_plein.shape)

(49461, 1528)


In [102]:
# print the name of the columns with a majority of zeros
col_maj_zero = []
for x in insee_plein.columns:
    if (insee_plein[x] == 0).sum() > 16400:
        col_maj_zero.append(x)

print(len(col_maj_zero))


907


> 907 colomnes sur 1528 avec plus d'un tiers de 0, je drop tout ça. Il restera donc 621 colomnes

In [103]:
insee_plein.drop(columns=col_maj_zero, inplace=True)

In [105]:
# print the name of the columns with a majority of nan values
col_maj_nan = []
for x in insee_plein.columns:
    if (insee_plein[x].isnull()).sum() > 101:
        col_maj_nan.append(x)

print(len(col_maj_nan))

283


> On règle les problèmes de la colonne code postal (qui est pour l'instant une colomne de type "object), en supprimant les codes postaux contenant des lettres

In [110]:
insee_plein = insee_plein[~insee_plein["code_postal"].str.contains("A|B", na=False)]

insee_plein["code_postal"] = insee_plein["code_postal"].astype(int)

C:\Users\maila\AppData\Local\Temp\ipykernel_22984\1362031746.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  insee_plein["code_postal"] = insee_plein["code_postal"].astype(int)


> On a 0 colomnes avec 102 ou plus de NaN values, et 741 avec 101 valeurs nulles, donc on drop rien. En plus, après le drop des colomnes pleines de 0, on a déjà plus que 283 columnes avec 101 valeurs nulles

> On drop ensuite les colomnes de types "object"

In [111]:
# drop object columns
print(len(insee_plein.columns))
colomnes_object = []
for x in insee_plein.columns:
    if insee_plein[x].dtype == "object":
        colomnes_object.append(x)

insee_plein.drop(columns=colomnes_object, inplace=True)
print(len(insee_plein.columns))

621
601


C:\Users\maila\AppData\Local\Temp\ipykernel_22984\2930573705.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  insee_plein.drop(columns=colomnes_object, inplace=True)


> Il reste donc 601 colomnes

#### On s'occupe du cas où il y a plusieurs fois le même code postal d'écrit

In [114]:
print(insee_plein[["code_postal"]].duplicated().sum())

# drop duplicate rows
insee_plein = insee_plein.drop_duplicates(subset=["code_postal"], keep="first")

14115


> Pour un peu plus de lisibilité, on écrit les colonnes restantes dans un .txt

In [93]:
liste_col = insee_plein.columns

file = open('colomnes_INSEE.txt','w')
for col in liste_col:
	file.write(col+"\n")
file.close()

> Reste alors à merge

In [118]:
X = pd.read_csv("../Prix_metre/UX_avec_prix_et_images_complet_et_inflation.csv")

In [119]:
X = X.merge(insee_plein, how='left', left_on='postal_code', right_on='code_postal')
print(len(X))

46707


In [120]:
# print the number of duplicate rows in X_train
print(X[["id_annonce"]].duplicated().sum())

0


In [121]:
# drop duplicate rows
X = X.drop_duplicates(subset=["id_annonce"], keep="first")

In [125]:
# Nombres de valeurs manquantes
print(X.shape)

(46707, 656)


In [126]:
X.to_csv('essai_big_INSEE.csv', index=False)